<a href="https://colab.research.google.com/github/AlessandraSalanitri/DataScienceProject/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install mlxtend
!pip install mlxtend
!pip install --upgrade mlxtend
import pandas as pd

dataset = pd.read_csv('medicine_dataset.csv')

print("Dataset Preview:")
print(dataset.head())

<ipython-input-1-6470354a9f51>:6: DtypeWarning: Columns (42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('medicine_dataset.csv')


Dataset Preview:
   id                      name                     substitute0  \
0   1  augmentin 625 duo tablet  Penciclav 500 mg/125 mg Tablet   
1   2       azithral 500 tablet         Zithrocare 500mg Tablet   
2   3          ascoril ls syrup                 Solvin LS Syrup   
3   4      allegra 120mg tablet                    Lcfex Tablet   
4   5            avil 25 tablet              Eralet 25mg Tablet   

              substitute1              substitute2            substitute3  \
0  Moxikind-CV 625 Tablet  Moxiforce-CV 625 Tablet     Fightox 625 Tablet   
1         Azax 500 Tablet          Zady 500 Tablet  Cazithro 500mg Tablet   
2       Ambrodil-LX Syrup        Zerotuss XP Syrup         Capex LS Syrup   
3     Etofex 120mg Tablet     Nexofex 120mg Tablet    Fexise 120mg Tablet   
4                     NaN                      NaN                    NaN   

               substitute4 sideEffect0       sideEffect1     sideEffect2  ...  \
0  Novamox CV 625mg Tablet    Vomiti

In [2]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

#relevant columns
substitute_columns = ["substitute0", "substitute1", "substitute2", "substitute3", "substitute4"]
usage_columns = ["use0", "use1", "use2", "use3", "use4"]
side_effect_columns = [f"sideEffect{i}" for i in range(42)]

#selected columns
columns_of_interest = substitute_columns + usage_columns + side_effect_columns
data = dataset[columns_of_interest]

#Handle missing values
data.fillna("Unknown", inplace=True)

#Convert data to transaction format
transactions = data.apply(lambda row: [item for item in row if item != "Unknown"], axis=1)
transactions = transactions.tolist()

#transactions for Apriori analysis
with open("transactions.txt", "w") as f:
    for transaction in transactions:
        f.write(",".join(transaction) + "\n")

#processed dataset saved as new dataset
processed_data_path = "processed_medicine_data.csv"
data.to_csv(processed_data_path, index=False)
print(f"Processed dataset saved as {processed_data_path}")

#display a sample
print("Transaction Data Sample:")
for i, transaction in enumerate(transactions[:5]):
    print(f"Transaction {i + 1}: {transaction}")


<ipython-input-2-f558a2610252>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna("Unknown", inplace=True)


Processed dataset saved as processed_medicine_data.csv
Transaction Data Sample:
Transaction 1: ['Penciclav 500 mg/125 mg Tablet', 'Moxikind-CV 625 Tablet', 'Moxiforce-CV 625 Tablet', 'Fightox 625 Tablet', 'Novamox CV 625mg Tablet', 'Treatment of Bacterial infections', 'Vomiting', 'Nausea', 'Diarrhea']
Transaction 2: ['Zithrocare 500mg Tablet', 'Azax 500 Tablet', 'Zady 500 Tablet', 'Cazithro 500mg Tablet', 'Trulimax 500mg Tablet', 'Treatment of Bacterial infections', 'Vomiting', 'Nausea', 'Abdominal pain', 'Diarrhea']
Transaction 3: ['Solvin LS Syrup', 'Ambrodil-LX Syrup', 'Zerotuss XP Syrup', 'Capex LS Syrup', 'Broxum LS Syrup', 'Treatment of Cough with mucus', 'Nausea', 'Vomiting', 'Diarrhea', 'Upset stomach', 'Stomach pain', 'Allergic reaction', 'Dizziness', 'Headache', 'Rash', 'Hives', 'Tremors', 'Palpitations', 'Muscle cramp', 'Increased heart rate']
Transaction 4: ['Lcfex Tablet', 'Etofex 120mg Tablet', 'Nexofex 120mg Tablet', 'Fexise 120mg Tablet', 'Histafree 120 Tablet', 'Treatm

In [3]:
#transactions to proper format
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
apriori_df = pd.DataFrame(te_ary, columns=te.columns_)

#find frequent itemsets with apriori
frequent_itemsets = apriori(apriori_df, min_support=0.05, use_colnames=True)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Extract association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)


# results
print("\nFrequent Itemsets:")
print(frequent_itemsets.head())

print("\nAssociation Rules:")
print(rules.head())


Frequent Itemsets:
    support             itemsets
0  0.094276       ( Pain relief)
1  0.128319     (Abdominal pain)
2  0.072037  (Allergic reaction)
3  0.050601     (Blurred vision)
4  0.117622       (Constipation)

Association Rules:
      antecedents     consequents  antecedent support  consequent support  \
0  ( Pain relief)      (Diarrhea)            0.094276            0.565209   
1      (Diarrhea)  ( Pain relief)            0.565209            0.094276   
2  ( Pain relief)        (Nausea)            0.094276            0.634233   
3        (Nausea)  ( Pain relief)            0.634233            0.094276   
4  ( Pain relief)      (Vomiting)            0.094276            0.404205   

    support  confidence      lift  representativity  leverage  conviction  \
0  0.066534    0.705739  1.248634               1.0  0.013249    1.477570   
1  0.066534    0.117716  1.248634               1.0  0.013249    1.026568   
2  0.082162    0.871501  1.374103               1.0  0.022369    2.8

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
